In [193]:
%load_ext autoreload
%autoreload complete

from string_compression import ICFP, Human
import lex

import math
from pathlib import Path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [215]:
x = lex.parse_int('x')
vx = ICFP.var(x)
lx = ICFP.lam(x)

y = lex.parse_int('y')
vy = ICFP.var(y)

z = lex.parse_int('z')
vz = ICFP.var(z)

u = lex.parse_int('u')
vu = ICFP.var(u)

w = lex.parse_int('w')
vw = ICFP.var(w)

f = lex.parse_int('f')
vf = ICFP.var(f)
lf = ICFP.lam(f)

#'x', 'y', 'z', 'w', 'u', 'v', 's', 't', 'a', 'b', 'c', 'd'

In [3]:
def concat(*strs: ICFP) -> ICFP:
    if len(strs) == 0:
        return ICFP.str('')

    if len(strs) == 0:
        return strs[0]

    result = [
        f'B. {strs[i]} '
        for i in range(len(strs) - 1)
    ] + [
        strs[-1].value
    ]

    return ICFP(
        ''.join(result)
    )

In [4]:
concat(
    ICFP.str('1'),
    ICFP.str('22'),
    ICFP.str('333'),
    #vx,
).info()

[I]: B. SV B. SWW SXXX
[-]: '1' ++ ('22' ++ '333')
[O]: 122333


In [10]:
def lambda_concat_times(var_index: int, times: int) -> ICFP:
    lam = ICFP.lam(var_index)
    var = ICFP.var(var_index)

    return lam + concat(*[var] * times)

In [11]:
lambda_concat_times(x, 4).info()

[I]: Lx B. vx B. vx B. vx vx
[-]: λ d. (d ++ (d ++ (d ++ d)))
[O]: <function eval_tokens.<locals>.<lambda> at 0x724bedb4f740>


In [7]:
def apply_lambda(lam: ICFP, arg: ICFP) -> ICFP:
    return ICFP.apply() + lam + arg

In [12]:
apply_lambda(
    lambda_concat_times(x, 4),
    ICFP.str('hi')
).info()

[I]: B$ Lx B. vx B. vx B. vx vx S()
[-]: (λ d. (d ++ (d ++ (d ++ d))))('hi')
[O]: hihihihi


In [15]:
def repeat_apply(lam: ICFP, initial_arg: ICFP, times: int) -> ICFP:
    return ICFP(f'{ICFP.apply()} {lam} ' * (times) + initial_arg.value)

In [16]:
repeat_apply(
    lambda_concat_times(x, 4),
    ICFP.str('LOL'),
    2,
).info()

[I]: B$ Lx B. vx B. vx B. vx vx B$ Lx B. vx B. vx B. vx vx SFIF
[-]: (λ d. (d ++ (d ++ (d ++ d))))((λ d. (d ++ (d ++ (d ++ d))))('LOL'))
[O]: LOLLOLLOLLOLLOLLOLLOLLOLLOLLOLLOLLOLLOLLOLLOLLOL


In [205]:
repeat_apply(
    lambda_concat_times(x, 3),
    ICFP.str('Ldd'),
    5,
)

B$ Lx B. vx B. vx vx B$ Lx B. vx B. vx vx B$ Lx B. vx B. vx vx B$ Lx B. vx B. vx vx B$ Lx B. vx B. vx vx SF$$

In [17]:
def lambda_repeat_apply(var_index: int, apply_lam: ICFP, times: int) -> ICFP:
    lam = ICFP.lam(var_index)
    var = ICFP.var(var_index)
    body = repeat_apply(apply_lam, var, times)

    return lam + body


In [18]:
lambda_repeat_apply(x, vy, 3).info()

[I]: Lx B$ vy B$ vy B$ vy vx
[-]: λ d. (c(c(c(d))))
[O]: <function eval_tokens.<locals>.<lambda> at 0x724beec11940>


In [27]:
def let(lhs_index: int, rhs: ICFP, expr: ICFP) -> ICFP:
    lam = ICFP.lam(lhs_index)
    assert ICFP.var(lhs_index).value in expr.value

    return apply_lambda(
        lam + expr,
        rhs,
    )

In [29]:
let(x, ICFP.str('hi'),
    concat(vx, vx)
).info()

[I]: B$ Lx B. vx vx S()
[-]: (λ d. (d ++ d))('hi')
[O]: hihi


In [62]:
def op_equal(lhs: ICFP, rhs: ICFP) -> ICFP:
    return ICFP.eq_() + lhs + rhs

def op_if(condition: ICFP, if_true: ICFP, if_false: ICFP) -> ICFP:
    return ICFP.if_() + condition + if_true + if_false

def op_sub(lhs: ICFP, rhs: ICFP) -> ICFP:
    return ICFP('B-') + lhs + rhs

In [38]:
op_if(
    op_equal(ICFP.str('hi'), ICFP.str('i')),
    ICFP.int(1),
    ICFP.int(0),
).info()

[I]: ? B= S() S) I" I!
[-]: if ('hi' == 'i') { 1 } else { 0 }
[O]: 0


In [80]:
def y_combinator(self_lambda_index: int, lam: ICFP):
    assert ICFP.var(self_lambda_index).value in lam.value

    return ICFP(f'B$ L" B$ L# B$ v" B$ v# v# L# B$ v" B$ v# v# {ICFP.lam(self_lambda_index)} ' + lam.value)


In [83]:
apply_lambda(
    y_combinator(f,
        lx + op_if(op_equal(vx, ICFP.int(0)),
            ICFP.str('zero'),
            concat(
                apply_lambda(vf, op_sub(vx, ICFP.int(1))),
                ICFP.str('iter'),
            )
        )
    ),
    ICFP.int(2),
).info()

[I]: B$ B$ L" B$ L# B$ v" B$ v# v# L# B$ v" B$ v# v# Lf Lx ? B= vx I! S:%2/ B. B$ vf B- vx I" S)4%2 I#
[-]: ((λ d. ((λ c. (d(c(c))))(λ c. (d(c(c))))))(λ b. (λ a. (if (a == 0) { 'zero' } else { (b(a - 1)) ++ 'iter' }))))(2)
[O]: zeroiteriter


In [99]:
def repeate_str_recursive(s: ICFP, times: int, initial: ICFP | None = None):
    return apply_lambda(
        y_combinator(f,
            ICFP.lam(w) + op_if(op_equal(vw, ICFP.int(0)),
                initial if initial is not None else ICFP.str(''),
                concat(
                    apply_lambda(vf, op_sub(vw, ICFP.int(1))),
                    s,
                )
            )
        ),
        ICFP.int(times),
    )

In [100]:
repeate_str_recursive(ICFP.str('hi'), 1000).info()

[I]: B$ B$ L" B$ L# B$ v" B$ v# v# L# B$ v" B$ v# v# Lf Lw ? B= vw I! S B. B$ vf B- vw I" S() I+]
[-]: ((λ d. ((λ c. (d(c(c))))(λ c. (d(c(c))))))(λ b. (λ a. (if (a == 0) { '' } else { (b(a - 1)) ++ 'hi' }))))(1000)
[O]: hihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihihi

In [101]:
def lambda_repate_recursive(var_index: int, times: int) -> ICFP:
    lam = ICFP.lam(var_index)
    var = ICFP.var(var_index)

    return lam + apply_lambda(
        y_combinator(f,
            ICFP.lam(w) + op_if(op_equal(vw, ICFP.int(0)),
                var,
                concat(
                    apply_lambda(vf, op_sub(vw, ICFP.int(1))),
                    var,
                )
            )
        ),
        ICFP.int(times - 1),
    )

In [103]:
apply_lambda(
    lambda_repate_recursive(y, 10),
    ICFP.str('hi')
).info()

[I]: B$ Ly B$ B$ L" B$ L# B$ v" B$ v# v# L# B$ v" B$ v# v# Lf Lw ? B= vw I! vy B. B$ vf B- vw I" vy I* S()
[-]: (λ d. (((λ c. ((λ b. (c(b(b))))(λ b. (c(b(b))))))(λ a. (λ t. (if (t == 0) { d } else { (a(t - 1)) ++ d }))))(9)))('hi')
[O]: hihihihihihihihihihi


In [118]:
def lambda_repeat_at_least(var_index: int, times: int) -> ICFP:
    alternatives = [
        lambda_concat_times(var_index, times),
        lambda_repate_recursive(var_index, times),
    ]

    for concat_times in range(1, math.isqrt(times) + 2):
        concat = lambda_concat_times(11, concat_times)

        for repeat_apply_times in range(2, times + 1):
            if concat_times ** repeat_apply_times < times:
                continue

            without_let = lambda_repeat_apply(var_index, concat, repeat_apply_times)
            with_let = let(
                u, concat,
                lambda_repeat_apply(var_index, vu, repeat_apply_times),
            )

            alternatives += [without_let, with_let]

            break

    return min(
        alternatives,
        key=lambda a: len(a.value)
    )


In [125]:
lambda_repeat_at_least(x, )

Lx B$ B$ L" B$ L# B$ v" B$ v# v# L# B$ v" B$ v# v# Lf Lw ? B= vw I! vx B. B$ vf B- vw I" vx I%sB*

In [217]:
def repeate_string_with_base_exp_at_least(s: str, times: int):
    alternatives = []
    for base in range(math.isqrt(times) + 1):
        for exp in range(2, times):
            if base ** exp >= times:
                alternatives.append(
                    apply_lambda(
                        lf + repeat_apply(
                            vf,
                            ICFP.str(s),
                            exp
                        ),
                        lambda_concat_times(x, base)
                    )
                )
                break

    if len(alternatives) == 0:
        print(f'{times} times {len(s)} len str {s}')

    return min(
        alternatives,
        key=lambda a: len(a.value)
    )


def repeate_string_at_least(s: str, times: int) -> ICFP:
    smallest_string = s * times
    alternatives = [
        ICFP.str(smallest_string),
        Human(smallest_string).div_mod_compact(),
    ]

    for i in range(1, times + 1):
        new_s = s * i
        new_t = math.ceil(len(smallest_string) / len(new_s))

        alternatives.append(
            apply_lambda(
                lambda_repeat_at_least(x, new_t),
                ICFP.str(new_s)
            )
        )


        if new_t > 3:
            alternatives.append(
                repeate_string_with_base_exp_at_least(new_s, new_t)
            )


    return min(
        alternatives,
        key=lambda a: len(a.value)
    )

In [222]:
repeate_string_at_least('A', 1000).info()

[I]: B$ Lf B$ vf B$ vf B$ vf B$ vf S;;;; Lx B. vx B. vx B. vx vx
[-]: (λ d. (d(d(d(d('AAAA'))))))(λ c. (c ++ (c ++ (c ++ c))))
[O]: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

In [223]:
lambdaman6 = ICFP(Path('solutions/lambdaman/compact/6_73').read_text())
lambdaman6.info()

[I]: B. S3/,6%},!-"$!-!.[} B$ Lf B$ vf B$ vf B$ vf SLLLLLLLL Lx B. vx B. vx vx
[-]: 'solve lambdaman6 ' ++ ((λ d. (d(d(d('RRRRRRRR')))))(λ c. (c ++ (c ++ c))))
[O]: solve lambdaman6 RRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR


In [226]:
repeate_string_at_least('A', 199).info()

[51]: B$ Lf B$ vf B$ vf B$ vf S;;;;;;;; Lx B. vx B. vx vx
[-]: (λ d. (d(d(d('AAAAAAAA')))))(λ c. (c ++ (c ++ c)))
[O]: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA


In [227]:
concat(
    ICFP.str('solve lambdaman6 '),
    repeate_string_at_least('A', 199),
).info()

[73]: B. S3/,6%},!-"$!-!.[} B$ Lf B$ vf B$ vf B$ vf S;;;;;;;; Lx B. vx B. vx vx
[-]: 'solve lambdaman6 ' ++ ((λ d. (d(d(d('AAAAAAAA')))))(λ c. (c ++ (c ++ c))))
[O]: solve lambdaman6 AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA


In [169]:
repeate_string_with_base_exp('AA', int(1000 / 2))

B$ Lj B. vj B. vj vj B$ Lj B. vj B. vj vj B$ Lj B. vj B. vj vj B$ Lj B. vj B. vj vj B$ Lj B. vj B. vj vj B$ Lj B. vj B. vj vj S;;

In [159]:
find_all_pow(100)

[[2, 7],
 [3, 5],
 [4, 4],
 [5, 3],
 [6, 3],
 [7, 3],
 [8, 3],
 [9, 3],
 [10, 2],
 [11, 2]]

In [ ]:
def pow_string(s: z.ICFPString, base: int, exponent: int) -> z.ICFPString:
    if base == 1:
        return s

    product = 'Lx' + ' B. vx' * (base - 1) + ' vx'
    exp = 'Lf' + ' B$ vf' * exponent

    return z.ICFPString(f'B$ {exp} {s.value} {product}')


def repeat_string_at_least(s: z.HumanString, k: int):
    assert 0 < k

    sqrt = math.isqrt(k)

    alternatives = [
        z.HumanString(s.value * k).to_icfp(),
        apply_lambda(product_lambda(k, 0), s.to_icfp().value),
    ] + [
        pow_string(s.to_icfp(), base, exp, 0, 1)
        for base, exp in [(sqrt, sqrt), (sqrt, sqrt + 1), (sqrt + 1, sqrt)]
        if base ** exp >= k
    ]

    return alternatives

In [ ]:
s = '2'
times = 10

for i in range(1, times + 1):
    new_s = s * i
    new_times =

In [127]:
'134'[:1]

'1'

In [ ]:
def y_combinator_repeat(expr: ICFP, times: int):


In [52]:
ICFP.int(0)

I!

In [ ]:
y_combinator(f,
    f'{ICFP.lam(x)} ? B= {vx} {ICFP.int(0)} {ICFP.str("finish")} '
)

In [ ]:
'B$ B$ L" B$ L# B$ v" B$ v# v# L# B$ v" B$ v# v# L" '

In [45]:
alternatives[0].info()

[I]: L. B. B. B. B. v. v. v. v. v.
[-]: λ d. ((((d ++ d) ++ d) ++ d) ++ d)
[O]: <function eval_tokens.<locals>.<lambda> at 0x73169e7900e0>


In [58]:
lex.parse_int('x')

87

In [60]:
alternatives[2]

ICFP(B$ B$ L3 L. B$ v3 B$ v3 B$ v3 v. L, B. v, v,)

In [20]:
apply_lambda(lambda_sum_concat(1, 4), z.ICFP.str('hi')).info()

B$ L" B. B. B. v" v" v" v" S()

(λ d. (((d ++ d) ++ d) ++ d))('hi')

hihihihi


In [ ]:
def lambda_repeat_lambda

In [ ]:
exp = 'Lf' + ' B$ vf' * exponent

In [ ]:
def product_lambda(k: int, var_index: int):
    assert 0 <= var_index
    var = lex.encode_int(var_index)

    return f'L{var} ' + sum(
        (
            z.ICFPString(f'v{var}')
            for _ in range(k - 1)
        ),
        start=z.ICFPString(f'v{var}')
    ).value

In [22]:
def concat_strings(*strs: z.ICFPString) -> z.ICFPString:
    result = [
        f'B. {strs[i].value} '
        for i in range(len(strs) - 1)
    ]

    result.append(strs[-1].value)

    return z.ICFPString(
            ''.join(
            result
        )
    )


In [23]:
(z.HumanString('LOVE').to_icfp() + z.HumanString(' ').to_icfp() + z.HumanString('YOU').to_icfp()).to_human()

HumanString(LOVE YOU)

In [24]:
concat_strings(
    z.HumanString('LOVE').to_icfp(),
    z.HumanString(' ').to_icfp(),
    z.HumanString('YOU').to_icfp(),
    z.HumanString('!!').to_icfp(),
).to_human()

HumanString(LOVE YOU!!)

In [25]:
def pow_string(s: z.ICFPString, base: int, exponent: int) -> z.ICFPString:
    if base == 1:
        return s

    product = 'Lx' + ' B. vx' * (base - 1) + ' vx'
    exp = 'Lf' + ' B$ vf' * exponent

    return z.ICFPString(f'B$ {exp} {s.value} {product}')


In [26]:
pow_string(
    z.HumanString('LOVE').to_icfp(),
    2,
    8,
).to_human()

HumanString(LOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVELOVE

In [27]:
def product_lambda(k: int, var_index: int):
    assert 0 <= var_index
    var = lex.encode_int(var_index)

    return f'L{var} ' + sum(
        (
            z.ICFPString(f'v{var}')
            for _ in range(k - 1)
        ),
        start=z.ICFPString(f'v{var}')
    ).value

In [28]:
def apply_lambda(lam: str, arg: str) -> z.ICFPString:
    return z.ICFPString(
        f'B$ {lam} {arg}'
    )

In [34]:
def lambda_repeat_lambda_with_arg(lambda_var_index: int, initial_arg: str, repeat: int):
    assert 1 < repeat

    var = lex.encode_int(lambda_var_index)

    return f'L{var}' + f' B$ v{var}' * repeat + f' {initial_arg}'

In [30]:
def pow_string(s: z.ICFPString, base: int, exponent: int, base_var_index: int, exponent_var_index: int):
    print(f'{base} ** {exponent}')
    p = product_lambda(base, base_var_index)
    exp = lambda_repeat_lambda_with_arg(exponent_var_index, s, exponent)
    return apply_lambda(exp, p)

In [31]:
def Y_combinator(stop_condition: z.ICFPString, ) -> z.ICFPString:

    f'''B$ B$ L" B$ L# B$ v" B$ v# v# L# B$ v" B$ v# v# L" L# ? B= v# I! S B. B$ v" B/ v# I{alphabet_size_encoded} BT I" BD B% v# I{alphabet_size_encoded} S{alphabet_encoded} I{compact_encoded}'''

In [ ]:
def lambda_times(product: int, pow: int):
    p = product_lambda(product, 7)
    exp = lambda_repeat_lambda_with_arg(5, )
    return ''

In [19]:
product_lambda(3, 0)

'L! B. B. v! v! v!'

In [ ]:
def

In [ ]:
n = 98
sol = ('D' * n + 'L' * n + 'U' * n + 'R' * n) * 25

In [20]:
math.isqrt(10)

3

In [32]:
def repeat_string_at_least(s: z.HumanString, k: int):
    assert 0 < k

    sqrt = math.isqrt(k)

    alternatives = [
        z.HumanString(s.value * k).to_icfp(),
        apply_lambda(product_lambda(k, 0), s.to_icfp().value),
    ] + [
        pow_string(s.to_icfp(), base, exp, 0, 1)
        for base, exp in [(sqrt, sqrt), (sqrt, sqrt + 1), (sqrt + 1, sqrt)]
        if base ** exp >= k
    ]

    return alternatives

In [37]:
199 / 8

24.875

In [38]:
for x in repeat_string_at_least(
    z.HumanString('RRRRRRRR'),
    25,
):
    print(f'{len(x.value)} {x}  ', end='')
    print()
    #last = x
    #h = x.to_human()
    #print(f'{len(h.value)} {h}')

5 ** 5
5 ** 6
6 ** 5
201 ICFPString(SLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL)  
162 ICFPString(B$ L! B. B. B. B. B. B. B. B. B. B. B. B. B. B. B. B. B. B. B. B. B. B. B. B. v! v! v! v! v! v! v! v! v! v! v! v! v! v! v! v! v! v! v! v! v! v! v! v! v! SLLLLLLLL)  
75 ICFPString(B$ L" B$ v" B$ v" B$ v" B$ v" B$ v" SLLLLLLLL L! B. B. B. B. v! v! v! v! v!)  
81 ICFPString(B$ L" B$ v" B$ v" B$ v" B$ v" B$ v" B$ v" SLLLLLLLL L! B. B. B. B. v! v! v! v! v!)  
81 ICFPString(B$ L" B$ v" B$ v" B$ v" B$ v" B$ v" SLLLLLLLL L! B. B. B. B. B. v! v! v! v! v! v!)  


In [42]:
for i in range(2, 10):
    for j in range(i, 10):
        print(f'{i} ** {j} = {i ** j}')

2 ** 2 = 4
2 ** 3 = 8
2 ** 4 = 16
2 ** 5 = 32
2 ** 6 = 64
2 ** 7 = 128
2 ** 8 = 256
2 ** 9 = 512
3 ** 3 = 27
3 ** 4 = 81
3 ** 5 = 243
3 ** 6 = 729
3 ** 7 = 2187
3 ** 8 = 6561
3 ** 9 = 19683
4 ** 4 = 256
4 ** 5 = 1024
4 ** 6 = 4096
4 ** 7 = 16384
4 ** 8 = 65536
4 ** 9 = 262144
5 ** 5 = 3125
5 ** 6 = 15625
5 ** 7 = 78125
5 ** 8 = 390625
5 ** 9 = 1953125
6 ** 6 = 46656
6 ** 7 = 279936
6 ** 8 = 1679616
6 ** 9 = 10077696
7 ** 7 = 823543
7 ** 8 = 5764801
7 ** 9 = 40353607
8 ** 8 = 16777216
8 ** 9 = 134217728
9 ** 9 = 387420489


In [39]:
3 ** 3

27

In [19]:
last

ICFPString(B$ L! B. B. B. B. B. v! v! v! v! v! v! ICFPString(SFVWXY+$3))

In [22]:
last.to_human()

HumanString(6369806710782337158020139004391515523328)

In [49]:
human_string

6369806710782337158020139004391515523328

In [ ]:
pow_string(
    z.HumanString('FUCK').to_icfp(),
    2,
    8,
).to_human()

HumanString(FUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCK

In [46]:
math.isqrt(6)

2

In [ ]:
'Lf B$ vf B$ vf B$ vf S@O=E'

In [25]:
z.ICFPString('B$ Lf B$ vfB$ vfB$ vf S@O=E Lx B. vxB. vx vx')
z.ICFPString('B$ Lf B$ vf B$ vf B$ vf S@O=E Lx B. vx B. vx vx').to_human()

HumanString(FUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCK)

In [4]:
s6 = z.ICFPString.from_file('solutions/lambdaman/compact/6_73')
s6

ICFPString(B. S3/,6%},!-"$!-!.[} B$ Lf B$ vf B$ vf B$ vf SLLLLLLLL Lx B. vx B. vx vx)

In [6]:
s6.pprint()

"'solve lambdaman6 ' ++ ((λ d. (d(d(d('RRRRRRRR')))))(λ c. (c ++ (c ++ c))))"

In [5]:
s6.to_human()

HumanString(solve lambdaman6 RRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR)

In [7]:
rrr = 'RRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR'

In [8]:
len(rrr)

216

In [10]:
len(3 * 3 * 3 * 'RRRRRRRR')

216

In [2]:
s = z.HumanString('UDLLLDURRRRRURR')
s

HumanString(UDLLLDURRRRRURR)

In [4]:
s.div_mod_compact().to_human()

HumanString(UDLLLDURRRRRURR)

In [7]:
s.to_icfp()

ICFPString(SO>FFF>OLLLLLOLL)

In [3]:
s = 'UDLLLDURRRRRURR'
print(z.convert_string(s))

SO>FFF>OLLLLLOLL


In [4]:
z.compact_string(s)

'B$ B$ L" B$ L# B$ v" B$ v# v# L# B$ v" B$ v# v# L" L# ? B= v# I! S B. B$ v" B/ v# I% BT I" BD B% v# I% SFL>O I-%&2$'

In [6]:
z.eval_expr(z.compact_string(s))

'UDLLLDURRRRRURR'

In [146]:
from pathlib import Path

text = Path('solutions/3d/7.solution').read_text()

In [148]:
len(text)

682

In [151]:
len(Human(text).div_mod_compact().value)

584

In [5]:
import lex
import eval
import math
import com
import time

from pathlib import Path

In [6]:
def reproduce_int_sequence(i: int, alphabet_size: int):
    result = []
    while i != 0:
        d, m = divmod(i, alphabet_size)
        result.append(m)
        i = d

    result.reverse()

    return result

def compact_int_sequence(sequence: list[int], alphabet_size: int):
    it = iter(sequence)
    result = next(it)
    for s in it:
        result = result * alphabet_size + s

    return result


def create_alphabet(sequence: list[str]):
    alphabet = list(set(sequence))
    if len(alphabet) == 1:
        assert '*' not in alphabet
        alphabet.append('*')

    if alphabet.index(sequence[0]) == 0:
        alphabet.append(alphabet.pop(0))

    return alphabet

def compact_sequence(sequence: list[str]) -> tuple[int, list[str]]:
    alphabet = create_alphabet(sequence)

    result = compact_int_sequence(
        (
            alphabet.index(s)
            for s in sequence
        ),
        len(alphabet)
    )

    return result, alphabet


def parse_compact_sequence(compact: int, alphabet: list[str]):
    return [
        alphabet[i]
        for i in reproduce_int_sequence(compact, len(alphabet))
    ]

def compact_to_expr(compact: int, alphabet: list[str]):
    alphabet_size = len(alphabet)

    compact_encoded = lex.encode_int(compact)
    alphabet_encoded = lex.encode_string(alphabet)
    alphabet_size_encoded = lex.encode_int(alphabet_size)

    return f'''B$ B$ L" B$ L# B$ v" B$ v# v# L# B$ v" B$ v# v# L" L# ? B= v# I! S B. B$ v" B/ v# I{alphabet_size_encoded} BT I" BD B% v# I{alphabet_size_encoded} S{alphabet_encoded} I{compact_encoded}'''


def compact_string(string: str):
    compact, alphabet = compact_sequence(string)

    return compact_to_expr(compact, alphabet)


def prepare_lambdaman_result(task_i: int, expr):
    prefix = f'solve lambdaman{task_i} '
    prefix_encoded = lex.encode_string(prefix)

    return f'B. S{prefix_encoded} {expr}'


In [11]:
destination_dir = Path(f'solutions/lambdaman/raw')
#reported_destination_dir = destination_dir / 'reported'

def find_best_soultuion(i: int):
    solutions = list(destination_dir.glob(f'{i}_*'))
    if len(solutions) == 0:
        print(f'Lambdaman #{i} without solutions')
        return None

    best = min(solutions, key=lambda f: int(f.name.split('_')[1]))
    return best

def find_best_soultuion_compact(i: int):
    solutions = list(destination_dir.joinpath('compact').glob(f'{i}_*'))
    if len(solutions) == 0:
        print(f'Lambdaman #{i} without solutions')
        return None

    best = min(solutions, key=lambda f: int(f.name.split('_')[1]))
    return best


def get_best_solution_full(i: int):
    best = find_best_soultuion(i)

    if best is None:
        return None

    path = best.read_text().rstrip()field = L8.apply_solution(sol)

    return 'S' + lex.encode_string(f'solve lambdaman{i} {path}')


def get_best_solution_compact(i: int):
    best = find_best_soultuion_compact(i)

    if best is None:
        return None

    result = best.read_text().rstrip()

    return result


In [12]:
for i in range(1, 22):
    sol_path = find_best_soultuion(i)
    if sol_path is None:
        print(f'no solution for {i}')
        continue

    sol = sol_path.read_text().rstrip()

    compact = prepare_lambdaman_result(i, compact_string(sol))

    destination = Path(f'solutions/lambdaman/') / 'compact' / f'{i}_{len(compact)}'
    if destination.exists():
        print(f'skip {i}')
    else:
        destination.write_text(compact)

skip 1
skip 2
skip 3
skip 4
skip 5
skip 6
skip 7
skip 8
skip 9
skip 10
skip 12
skip 13
skip 14
skip 15
skip 16
skip 17
skip 18
skip 19


In [5]:
#for i in range(1, 22):
for i in [21]:
    best_full = get_best_solution_full(i)
    best_compact = get_best_solution_compact(i)

    if best_full is None and best_compact is None:
        continue

    best = min(
        [
            sol
            for sol in [best_full, best_compact]
            if sol is not None
        ]
        , key=lambda sol: len(sol)
    )

    print(f'{i} score: {len(best)} {best}')

    response_destination = Path(f'./solutions/lambdaman/reported/{i}_{len(best)}')
    response_destination.parent.mkdir(exist_ok=True)
    if response_destination.exists():
        print(f'skip {response_destination}')
    else:
        time.sleep(5)
        response = com.com(best, True)
        response_destination.write_text(response[0])
        print(response)


21 score: 10569 B. S3/,6%},!-"$!-!.WV} B$ B$ L" B$ L# B$ v" B$ v# v# L# B$ v" B$ v# v# L" L# ? B= v# I! S B. B$ v" B/ v# I% BT I" BD B% v# I% SOL>F I"50DaiivHA#{eaZhR5+Ho)P$Iru"xxfUs;&u?S(x5yo>D$_NE[HwPg<Y+7T|tq+]/%T]RxesZ@|yRu>.K98!+f6/;vnt$F"_~)TbUXh\")U7Zgelx)/}xxn_JHc99pg"2J)`_}N^r05wG#euEfVl[Z.G.3F.1U(~cc~mC#|uUr2|8n%)&mK!nAX~q5m:'Y7IhX10%-~OOJxhm|db)=#;Fs77EuRwEw\dS7ZHjKOUc|J!E58*<-L}tzXmERrFbSQ5i-toRUh'9@fogqVH^;cwJheK4%<9tW0p<)XtHH+0'dzS~exO`.o60T5nyxdWh2~EK_6YWL,4|t")~5{Z{HcK)Opsx{W.l?8^8d$~#w|S63Ma2V,1"Q(6H&/gn6FU7rze_D5K?UZ0v`f}akhLNjzQ)DIRa*S6WA.uKcvaFIDmQNfR65KR=9Z)"$+Wh8F|H)kmAG+Ur1Vfh-Tb"W?CoUE/q|mBWicIa\4e#yrKZu=;5zeH>Ad1[qyP0}tnSo[%sTAia+O@'^]J;KJ3pXuz1xHL%cf]?W"1fAP3J#$(T9(odn_:dX0eL9D\t2iWq$$A|\JpfXWgufT=q=),_Nb?fTL~Wvcm=S4e+Yh7iMr?!+yZ~}pKfQ(:#]$4(}fSd:oqmMIbNGh?)D@tphoicz?Oww[=vM2<zJ\`|qWaywl8l$LH.?D]qIpSjKsn;lL2vI@.\6]pkt-!d-Cp!eR%Bw/TaOF2v=3B)xj2Lrtfsw`i?Kwpi![]H$iV.P>O(pbOEBbKVqisNk@1R-MXEWnOG~\u|?A7'omo.kqT(~i#N3Z6YEO9Mw@4FPs>t9e\-5Oe=zk:1lSBy&92=',-.fjhS%f}[QO

In [18]:
sol_encoded = Path('solutions/lambdaman/compact/21_10563').read_text()

In [19]:
sol_encoded

'B. S3/,6%},!-"$!-!.WV} B$ B$ L" B$ L# B$ v" B$ v# v# L# B$ v" B$ v# v# L" L# ? B= v# I! S B. B$ v" B/ v# I% BT I" BD B% v# I% SLOF> I9E0\',&KM%1Z8c\\>MXuJbsL|?GU%iG@a/oWI;`c-~|dJ6TS<-#@IcEi_,/Z9vbZ2M~/_luY:&dhq6M{\'L<$oEte/EUaScW]w6(;qE"[PKv)oW1<?HV3B0@o\\|JQuBHq89@DsI7d"T?:6so;in;S/:*NkuDv+QHDdP*Svd8oeWc<?_DgH777vt,E~;zhx)gY$M5~pe;ekp%"5RB-BuH^bq@z2xuY.iWZ:-K6@b0}9h3y#X@LGZ"b2Y_)##}eu|\'.[PxI,&kVk^a?*Lt,+M"xaY?F;0<1W<POCQ7Pk?2PgQt8(Ah\'DE\'x\\$R@K.n\'i#jP>o5A>Q@ChX(:lul>cva>n<.^Pn(9L;a96$HW0`n\'5Ry__hSCU0y;-0m2Mn/}mK7-au).E}Apt(*MD|Jib2S/UdiZl#l;)@Tv|R}e{.}nLnmP.Y\'~:_Kr"Kxg)o\'B^TRqTi&|!H5x9.]]}zJ687I+NZkE|(8whY(IUQ8\\dB`2U8.W;Jq+^f5NRY|k0+j<;l+Od+$mDk@Jx]xl8F/y[jl7X.J}39dBlU-D&.NHOu#odN6ff,8F5Mt6iA\']CTs;R1g\\$vhD73_\\frw:iF^z42JI<k;zYgXc-XIgrx`Gvz4*dK/CInz(~G,}S4-Ch)ubccC[Ybs~=P30h;FgY}wYVA3\'fgy\'Jm#&QjnX),1}xQqn-6lW9!RH-.b^zH2F^3oGsJtim~`!R;NdX\\u%;-8MB\'F_Md\'qEFRYD$7A@qP2WY4/\']PXpgAtgL##"Ke@~9L3S1wB=#z^ZW(E.5DAxFB/f{m7;l1\'RSp\\zsv5Gbm\\Nko8ncV<yuMA9=Q_G0*SPggmxhu6kI&Qj0yrAte

In [20]:
com.com(sol_encoded, True)

['Correct, you solved lambdaman21 with a score of 10563!\n']